In [2]:
# ✅ Install required packages (Run this only once)
!pip install sounddevice scipy requests

# ✅ Import libraries
import os
from dotenv import load_dotenv

# Load variables from .env
load_dotenv()

import sounddevice as sd
from scipy.io.wavfile import write
import requests

# ✅ Your Hugging Face API Key (replace if needed)
# If you have stored the key in environment variables, keep os.getenv.
# Otherwise, just paste it directly like: HF_API_KEY = "hf_xxxxx"
HF_API_KEY = os.getenv("hugging_Face_API")  # or replace with your key: "hf_xxxxx"

# ✅ Function: Record audio
def record_audio(duration=5, filename="mic_recording.wav"):
    print(f"🎤 Recording for {duration} seconds...")
    fs = 16000  # Sample rate
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    write(filename, fs, audio)
    print(f"✅ Recording saved as {filename}")
    return filename

# ✅ Function: Transcribe audio using Hugging Face Whisper
def transcribe_with_huggingface(audio_path):
    url = "https://api-inference.huggingface.co/models/openai/whisper-large-v3"  # ✅ Updated model URL
    headers = {"Authorization": f"Bearer {HF_API_KEY}", "Content-Type": "audio/wav"}  # ✅ Set Content-Type

    with open(audio_path, "rb") as f:
        audio_data = f.read()  # ✅ Read audio data

        print("🔁 Sending audio to Hugging Face API...")
        response = requests.post(url, headers=headers, data=audio_data)  # ✅ Send raw data

    if response.status_code == 200:
        result = response.json()
        if "text" in result:
            return result["text"]
        else:
            print("⚠ No transcription found in response:", result)
            return None
    else:
        print("❌ Hugging Face Error:", response.status_code)
        print(response.text)
        return None

# ✅ Main program: Record & transcribe
file_path = record_audio(duration=5)
transcribed_text = transcribe_with_huggingface(file_path)

# ✅ Output
if transcribed_text:
    print("\n📝 Transcribed Text:\n", transcribed_text)
else:
    print("\n⚠ Transcription failed.")


🎤 Recording for 5 seconds...
✅ Recording saved as mic_recording.wav
🔁 Sending audio to Hugging Face API...

📝 Transcribed Text:
  Hello.
